# Analyse Hitzetage und Tropennächte

Dieses Notebook kann lokal oder **direkt im Browser** auf [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/meteotest/urban-heat-API-docs/add-heatdays-and-tropicalnights-analysis?labpath=python_data_analysis.ipynb) oder [![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/meteotest/urban-heat-API-docs/blob/add-heatdays-and-tropicalnights-analysis/python_data_analysis.ipynb) ausgeführt werden.

In [1]:
import requests 
import pandas as pd
import geopandas as gpd 
import matplotlib # for plotting
import folium
import branca

In [2]:
def get_stations() -> gpd.GeoDataFrame:
    response = requests.get(
            url=f"https://smart-urban-heat-map.ch/api/v2/latest"
    )
    stations = gpd.read_file(response.text)
    stations.drop(["temperature", "relativeHumidity", "dateObserved"], axis=1, inplace=True)
    
    return stations

def get_station_analysis(time_from: str, time_to: str) -> pd.DataFrame:
    stations = get_stations()
    
    stations['hitzetage'] = None
    stations['tropennaechte'] = None
    
    for idx, station in stations.iterrows():
        station_id = station.stationId
        response = requests.get(url=f"https://smart-urban-heat-map.ch/api/v2/timeseries?stationId={station_id}&timeFrom={time_from}&timeTo={time_to}")
        
        payload = response.json()
        if payload["values"] == []:
            continue
            
        df = pd.DataFrame(payload["values"])
    
        df["dateObserved"] = pd.to_datetime(df["dateObserved"])
        df["dateObserved"] = df["dateObserved"].dt.tz_convert("Europe/Zurich")
        
        # hier werden die Hitzetage und Tropennächte berechnet
        hitzetage = days_hotter_than(df, 30)
        tropennaechte = nights_hotter_than(df, 20)
        
        stations.at[idx, 'hitzetage'] = hitzetage
        stations.at[idx, 'tropennaechte'] = tropennaechte

    return stations

def days_hotter_than(df: pd.DataFrame, temperature: float) -> int:
    daily_maximums = df.groupby(
        df["dateObserved"].dt.date
    ).max()["temperature"]

    return (daily_maximums >= temperature).sum()

def nights_hotter_than(df: pd.DataFrame, temperature: float) -> int:
    df.loc[df['dateObserved'].dt.hour >= 18, 'nightDate'] = df["dateObserved"].dt.date + pd.Timedelta(days=1)
    df.loc[df['dateObserved'].dt.hour < 6, 'nightDate'] = df["dateObserved"].dt.date
        
    nightly_minimums = (
        df.groupby(df['nightDate'])['temperature']
       .min()
    )
        
    return (nightly_minimums > temperature).sum()

In [3]:
stations = get_stations()
stations

,stationId,name,geometry
0,11030,Galgenfeld Industrie,POINT (7.47194 46.95553)
1,11072,Rosengarten,POINT (7.46020 46.95188)
2,11024,Wankdorf ESP,POINT (7.46244 46.96798)
3,12008,Sportanlage Grien,POINT (7.29574 47.07509)
4,11100,Umland Bolligen,POINT (7.50376 46.96681)
...,...,...,...
95,12001,Mühleplatz Lyss,POINT (7.30895 47.07090)
96,11040,Galgenfeld Neubausiedlung,POINT (7.46987 46.95063)
97,11048,Bern Lorrainepark,POINT (7.44502 46.95565)
98,11013,Viererfeld 4 (Strasse Nord),POINT (7.44165 46.96523)


In [4]:
# get data of June
time_from = "2024-05-31T22:00:00Z"
time_to = "2024-06-30T22:00:00Z"

station_analysis = get_station_analysis(time_from, time_to)
station_analysis

,stationId,name,geometry,hitzetage,tropennaechte
0,11030,Galgenfeld Industrie,POINT (7.47194 46.95553),4,1
1,11072,Rosengarten,POINT (7.46020 46.95188),2,0
2,11024,Wankdorf ESP,POINT (7.46244 46.96798),0,1
3,12008,Sportanlage Grien,POINT (7.29574 47.07509),4,1
4,11100,Umland Bolligen,POINT (7.50376 46.96681),2,0
...,...,...,...,...,...
95,12001,Mühleplatz Lyss,POINT (7.30895 47.07090),4,1
96,11040,Galgenfeld Neubausiedlung,POINT (7.46987 46.95063),4,0
97,11048,Bern Lorrainepark,POINT (7.44502 46.95565),4,0
98,11013,Viererfeld 4 (Strasse Nord),POINT (7.44165 46.96523),2,0


In [5]:
# Hitzetage Map
m = folium.Map(location=[station_analysis.geometry.y.mean(), station_analysis.geometry.x.mean()], zoom_start=13, tiles="CartoDB positron")

colormap = branca.colormap.linear.YlOrRd_09
max_hitzetage = station_analysis.hitzetage.max()
colormap = colormap.scale(0, max_hitzetage).to_step(max_hitzetage) 
colormap.caption = "Anzahl Hitzetage"
colormap.add_to(m)

for idx, station in station_analysis.iterrows():
    folium.CircleMarker(
        location=(station.geometry.y, station.geometry.x),
        radius=5,
        color="black",
        weight=0.5,
        fill=True,
        fill_color=colormap(station.hitzetage),
        fill_opacity=1,
        popup=f"{station.hitzetage:.0f} Hitzetage",
        tooltip=f"{station['name']}: {station.hitzetage:.0f} Hitzetage",
    ).add_to(m)

m

In [6]:
# Tropennächte Map
m = folium.Map(location=[station_analysis.geometry.y.mean(), station_analysis.geometry.x.mean()], zoom_start=13, tiles="CartoDB positron")

colormap = branca.colormap.linear.YlOrRd_09
max_tropennaechte = station_analysis.tropennaechte.max() if station_analysis.tropennaechte.max() > 3 else 3
colormap = colormap.scale(0, max_tropennaechte).to_step(max_tropennaechte) 
colormap.caption = "Anzahl Tropennächte"
colormap.add_to(m)

for idx, station in station_analysis.iterrows():
    folium.CircleMarker(
        location=(station.geometry.y, station.geometry.x),
        radius=5,
        color="black",
        weight=0.5,
        fill=True,
        fill_color=colormap(station.tropennaechte),
        fill_opacity=1,
        tooltip=f"{station['name']}: {station.tropennaechte:.0f} Tropennächte",
    ).add_to(m)

m